Import

In [1]:
from Reader import Reader as rd
import numpy as np
import pandas as pd
from SingleImage import SingleImage as sg
import Camera

Inputting files and solving with all fmarks

In [2]:
camraw = rd.ReadCamFile('rc30.cam')
tagraw3574 = rd.Readtxtfile('fiducialsImg3574.txt')
tagraw3575 = rd.Readtxtfile('fiducialsImg3575.txt')
            
    
camera1 = Camera.Camera(camraw['f'], np.array([camraw['xp'],camraw['yp']]),\
                        np.array([camraw['k0'],camraw['k1'],camraw['k2']]),\
                        np.array([camraw['p1'],camraw['p2'],camraw['p3']]),\
                        camraw['fiducials'])

img1 = sg(camera1)
#inner_parameters, accuracies, residuals = img1.ComputeInnerOrientation(img_fmarks)
inner1 = img1.ComputeInnerOrientation(tagraw3574)
sigpost_2 = np.dot(inner1[1].T, inner1[1])/2
affinparams = pd.DataFrame(inner1[0]).rename(index={0:'a0',1:'b0',2:'a1',3:'a2',4:'b1',5:'b2'})
affinparams.columns = ['value(-)']
print(affinparams,'\n')
affinparams_v = pd.DataFrame(np.hstack((inner1[1][0::2],inner1[1][1::2]))\
                             ).rename(columns={0:'vx(pix)', 1:'vy(pix)'})
print(affinparams_v,'\n')
print('Sigma Post ^2 = ',sigpost_2[0][0])


       value(-)
a0  7923.100865
b0  7409.042691
a1    62.512580
a2    -0.037890
b1     0.041538
b2    62.515620 

    vx(pix)   vy(pix)
0  0.098925  0.189206
1 -0.009451  0.523313
2  0.523706 -0.065488
3 -0.249810  0.010083
4  0.265553 -0.401052
5  0.057375 -0.484770
6  0.090987  0.325732
7 -0.777285 -0.097025 

Sigma Post ^2 =  0.9291064529416714


Inputting files and solving without blunder fmarks

In [3]:
camraw = rd.ReadCamFile('rc30.cam')
tagraw3574 = rd.Readtxtfile('fiducialsImg3574.txt')
tagraw3575 = rd.Readtxtfile('fiducialsImg3575.txt')
            
    
camera1 = Camera.Camera(camraw['f'], np.array([camraw['xp'],camraw['yp']]),\
                        np.array([camraw['k0'],camraw['k1'],camraw['k2']]),\
                        np.array([camraw['p1'],camraw['p2'],camraw['p3']]),\
                        camraw['fiducials'][0:7,:])

img1 = sg(camera1)
#inner_parameters, accuracies, residuals = img1.ComputeInnerOrientation(img_fmarks)
inner1 = img1.ComputeInnerOrientation(tagraw3574[0:7,:])
sigpost_2 = np.dot(inner1[1].T, inner1[1])/2
affinparams = pd.DataFrame(inner1[0]).rename(index={0:'a0',1:'b0',2:'a1',3:'a2',4:'b1',5:'b2'})
affinparams.columns = ['value(-)']
print(affinparams,'\n')
affinparams_v = pd.DataFrame(np.hstack((inner1[1][0::2],inner1[1][1::2]))\
                             ).rename(columns={0:'vx(pix)', 1:'vy(pix)'})
print(affinparams_v,'\n')
print('Sigma Post ^2 = ',sigpost_2[0][0], '\n')
rms_x = pd.DataFrame(np.sqrt(np.diag((inner1[2]*sigpost_2[0][0]))))
rms_x = rms_x.rename(index={0:'a0(pix/mm)',1:'b0(pix/mm)',2:'a1',3:'a2',4:'b1',5:'b2'}, columns={0:'RMS'})
print(rms_x)

# affinparams.iloc[0,0]
# thetha = np.arctan(affinparams.iloc[4,0]/affinparams.iloc[5,0])
# print('thetha (rad)= ', thetha)
# tanGamma = (affinparams.iloc[2,0]*np.sin(thetha)+affinparams.iloc[3,0]*np.cos(thetha)) / \
#            (affinparams.iloc[4,0]*np.sin(thetha)+affinparams.iloc[5,0]*np.cos(thetha))
# print('tanGamma (-)= ', tanGamma)
# 
# sx = affinparams.iloc[2,0]*np.cos(thetha)-affinparams.iloc[3,0]*np.sin(thetha)
# sy = affinparams.iloc[2,0]*np.sin(thetha)+affinparams.iloc[3,0]*np.cos(thetha)/\
#      np.sin(np.arctan(tanGamma))
# print('Sx (-)= ', sx)
# print('Sy (-)= ', sy)

GeoParams = pd.DataFrame.from_dict(img1.ComputeGeometricParameters(inner1[0].flatten()), orient='index')
GeoParams = GeoParams.rename(columns={0:'Values'})
print('\n', GeoParams)

       value(-)
a0  7922.961246
b0  7409.025263
a1    62.514366
a2    -0.037890
b1     0.041761
b2    62.515620 

    vx(pix)   vy(pix)
0 -0.230051  0.148142
1  0.040278  0.529520
2  0.194738 -0.106551
3 -0.200075  0.016291
4  0.125926 -0.418481
5  0.117818 -0.477225
6 -0.048634  0.308304 

Sigma Post ^2 =  0.4882398107668194 

                 RMS
a0(pix/mm)  0.268313
b0(pix/mm)  0.268313
a1          0.002961
a2          0.002640
b1          0.002961
b2          0.002640

              Values
Dx      7922.961246
Dy      7409.025263
Thetha     0.000668
Gamma      0.000062
Sx        62.514378
Sy      -611.941856


In [4]:
camera1_inv = Camera.Camera(camraw['f'], np.array([camraw['xp'],camraw['yp']]),\
                        np.array([camraw['k0'],camraw['k1'],camraw['k2']]),\
                        np.array([camraw['p1'],camraw['p2'],camraw['p3']]),\
                        tagraw3574)

img1_inv = sg(camera1_inv)
#inner_parameters, accuracies, residuals = img1.ComputeInnerOrientation(img_fmarks)
inner1 = img1_inv.ComputeInnerOrientation(camraw['fiducials'])
sigpost_2 = np.dot(inner1[1].T, inner1[1])/2
affinparams = pd.DataFrame(inner1[0]).rename(index={0:'a0',1:'b0',2:'a1',3:'a2',4:'b1',5:'b2'})
affinparams.columns = ['value(-)']
print(affinparams,'\n')
affinparams_v = pd.DataFrame(np.hstack((inner1[1][0::2],inner1[1][1::2]))\
                             ).rename(columns={0:'vx(pix)', 1:'vy(pix)'})
print(affinparams_v,'\n')
print('Sigma Post ^2 = ',sigpost_2[0][0])

      value(-)
a0 -126.815885
b0 -118.430801
a1    0.015997
a2    0.000010
b1   -0.000011
b2    0.015996 

    vx(pix)   vy(pix)
0 -0.001584 -0.003026
1  0.000146 -0.008371
2 -0.008377  0.001053
3  0.003996 -0.000164
4 -0.004244  0.006418
5 -0.000914  0.007755
6 -0.001459 -0.005209
7  0.012435  0.001544 

Sigma Post ^2 =  0.00023774594632146903


In [5]:
imgtest = sg(camera1)
#inner_parameters, accuracies, residuals = img1.ComputeInnerOrientation(img_fmarks)
innertest = imgtest.ComputeInnerOrientation(tagraw3574[0:7,:])
# print(innertest[0].flatten())
# print(camraw['fiducials'])
picpoints = imgtest.CameraToImage(camraw['fiducials'], innertest[0].flatten())
print(picpoints)

[[ 1292.17194895 14031.3791418 ]
 [14545.28027777 14039.98252042]
 [ 1300.57973831   778.00544883]
 [14553.56292472   786.79629079]
 [ 7918.40492645 14410.89951937]
 [14924.25781757  7413.51477523]
 [ 7927.14236623   407.33830356]
 [  921.35202738  7404.66057384]]
